개인 공부 목적으로 BERT모델을 이용한 Baseline 코드에 주석을 한 줄씩 달아봤습니다.  


reference : https://www.kaggle.com/code/suraj520/beginner-friendly-bert  


For my own learning, I've provided Korean comments on each line of the baseline code

<div style="background-color:#5D73F2; color:#19180F; font-size:40px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px"> Beginner friendly approach </div>
<div style="background-color:#D5D9F2; color:#19180F; font-size:15px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px">
<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
BERT based approach. Know more about the architecture of BERT via <a href="https://www.kaggle.com/code/suraj520/bert-know-fit-infer"> kernel </a>   </div>
</div>


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Importing modules
    </div>

In [ ]:
# 모듈 임포트
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Loading the data    </div>

In [ ]:
# 데이터 로드
train_data = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
test_data = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Preprocessing the data    </div>

In [ ]:
tokenizer = BertTokenizer.from_pretrained('/kaggle/input/hugging-face-models-safe-tensors/bert-base-uncased')
"""
BertTokenizer는 Hugging Face의 transformers 라이브러리에서 제공하는 BERT 모델을 위한 토크나이저로, 텍스트를 모델이 처리할 수 있는 형태의 토큰으로 변환한다.
from_pretrained() : 특정 모델을 인자로 제공하면 해당 모델의 토크나이저 config 및 vocab을 로드할 수 있다.
"""



train_encodings = tokenizer.batch_encode_plus(
    train_data['text'].tolist(),
    truncation=True,
    padding=True
)                                 
"""
- batch_encode_plus는 여러개의 텍스트 문장을 한 번에 인코딩하는데 사용된다.
- tolist()는 pandas의 DataFrame이 제공하는 메서드로, 해당 데이터를 리스트로 변환한다.
- tolist()를 하는 이유: batch_encode_plus는 입력으로 Python 리스트, 튜플 또는 다른 시퀀스 형태의 데이터를 받을 수 있으므로
- turncation = True : 텍스트의 길이가 모델의 max length를 초과할 경우, 텍스트를 잘라낸다.
- padding = True : 인코딩된 텍스트가 동일한 길이를 갖도록 패딩을 추가한다.
"""

# test data에도 동일하게 적용
test_encodings = tokenizer.batch_encode_plus(
    test_data['text'].tolist(),
    truncation=True,
    padding=True
)


train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_data['content'].tolist()),
    torch.tensor(train_data['wording'].tolist())
)
"""
- torch.utils.data.TensorDataset : 텐서들의 데이터셋을 만든다.
- tokenizer.batch_encode_plus()를 사용해서 인코딩한 결과에서 'input_ids'를 가져온다. 
- 'input_ids'는 토큰화된 텍스를 나타내는 숫자 ID의 리스트이다.
- 'attention_mask'는 0,1로 구성돼있는데, 실제 토큰은 1, 패딩 토큰은 0의 값을 가진다.
- .tolist() : torch.tensor가 pandas의 Series객체를 직접 처리할 수 없어서 리스트로 변환
- wording도 마찬가지 
"""

# test data에도 동일하게 적용
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask'])
)


In [ ]:
# train_dataset 내용 확인

print("첫 번째 문장의 Inputs ids",train_dataset[0][0])
print("첫 번째 문장의 attention Mask", train_dataset[0][1])
print("첫 번째 문장의 'content'",train_dataset[0][2])
print("첫 번째 문장의 'wording'",train_dataset[0][3])

print("첫 번째 문장의 Inputs ids shape ",train_dataset[0][0].shape)
print("첫 번째 문장의 attention Mask shape", train_dataset[0][1].shape)


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Defining the BERT model    </div>

In [ ]:
class BERTModel(nn.Module):  # Pytorch의 nn.Module 클래스를 상속받아서 사용한다.
    def __init__(self):
        super(BERTModel, self).__init__()
        self.bert = BertModel.from_pretrained('/kaggle/input/hugging-face-models-safe-tensors/bert-base-uncased') # pretrain된 BERT 모델 로드

        self.dropout = nn.Dropout(0.1) # overfitting 방지를 위한 드롭아웃
        self.linear1 = nn.Linear(768, 256) # 첫 번째 선형 레이어. 입력 크기는 768(BERT 모델의 히든 레이어 크기), 출력 크기는 256
        self.linear2 = nn.Linear(256, 2)  # 두 번째 선형 레이어. 입력 크기는 256, 출력 크기는 2

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)  # input_ids와 attention_mask를 BERT모델에 전달하고, 그 출력은 outputs에 담는다.
        
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        """
        pooler_output이란? BERT의 마지막 Transformer 계층에서 [CLS] 토큰의 출력을 가져와서
        추가적인 Dense 레이어를 통과시킨 결과이다.
        이 CLS 토큰의 임베딩은 전체 입력 시퀀스의 문맥적 요약으로 사용된다.
        따라서 이 CLS 토큰의 임베딩(pooler_output)을 이용해서 추가 작업을 한다.
        """
        
        
        output = self.linear1(pooled_output) # 첫 번째 선형 레이어 통과
        output = nn.ReLU()(output)  # activation function인 ReLU 적용
        output = self.linear2(output)  # 두 번째 선형 레이어 통과
        return output


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Training the BERT model    </div>

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # CUDA가 사용 가능하면 device는 'cuda'로 설정되고, 아니면 'cpu'로 된다.
model = BERTModel().to(device) # 앞에서 정의한 BERTModel 클래스의 인스턴스인 model을 생성하고, .to(device)를 사용하여 생성된 모델을 device로 이동시킨다.
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5) # optimizer 정의. model.parameters()는 자동으로 모델 내의 모든 파라미터를 가져온다.
criterion = nn.MSELoss()  # loss function으로 Mean Squared Error 사용.


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Creating data loader and performing sanity check    </div>

In [ ]:
batch_size= 16  

In [ ]:
# Splitting training data into train and validation sets
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2, random_state=0)
# 데이터를 train용 데이터와 validation 데이터로 나눈다. train_dataset은 80%로, val_datset은 20%로


# Creating train loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Creating validation loader
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
"""
torch.utils.data.DataLoader는 Pytorch에서 제공하는 DataLoader
데이터 로더를 이용하면 데이터를 쉽게 배치로 분할하여 모델 train이나 validation에 사용할 수 있다.
일반적으로 train data는 shuffle을 하고, 검증, test는 shuffle을 안 한다.
"""


In [ ]:
for batch in train_loader:
    print(batch)
    
"""
여기서 각각의 batch가 뜻하는 것 : (input_ids, attention_mask, content, wording)
그래서 len(batch) 찍으면 모두 4가 나온다.
"""


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Training the model for 30 epochs    </div>

In [ ]:
# Training loop
model.train()  # model을 train 모드로 설정.
for epoch in range(30):  # 30 epoch동안 모델을 훈련한다
    running_loss = 0.0  # 현재 epoch에서 발생한 모든 배치의 loss값을 누적하기 위한 변수 running_loss
    for step, (input_ids, attention_mask, content, wording) in enumerate(train_loader):  # train_loader로부터 배치 데이터를 로드해서 각 배치에 대해서 아래의 코드를 실행
        
        # 해당 배치의 데이터를 GPU로 전송
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        content = content.to(device)
        wording = wording.to(device)

        optimizer.zero_grad()  # 이전 배치에서의 기울기 정보 초기화

        outputs = model(input_ids, attention_mask)  # 현재 배치의 data를 모델에 집어넣고, outputs을 받는다. outputs는 [batch_size, 2]가 될 것이다.
        
        loss = criterion(outputs[:, 0], content) + criterion(outputs[:, 1], wording) # outputs을 사용해서 loss 계산. 첫 번째 값을 content와, 두 번째 값을 wording과 비교한다.
        loss.backward()  # loss에 대한 gradient 계산
        optimizer.step()  # 계산된 gradient를 사용해서 parameter 업데이트
        
        if step % 500 == 0:  # 500 step마다 epoch, step, loss 출력
            print("Epoch {}, Step {}, Loss: {}".format(epoch+1, step, loss.item()))

        running_loss += loss.item()  # 현재 배치의 loss를 running_loss에 더한다.

    print(f"Epoch {epoch+1} Loss: {running_loss / len(train_loader)}")  # 각 epoch 이후에 평균 train loss를 출력

    # Validation loop
    model.eval()  # 모델을 validation 모드로 설정
    with torch.no_grad():  # gardient 계산 중지 (validation이므로)
        val_loss = 0.0  # train에서와 동일하게 진행, 근데 parameter 업데이트 과정이 없음
        
        # validation  data의 각 배치에 대해서 실행
        for val_step, (input_ids, attention_mask, content, wording) in enumerate(val_loader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            content = content.to(device)
            wording = wording.to(device)

            val_outputs = model(input_ids, attention_mask)
            val_loss += criterion(val_outputs[:, 0], content) + criterion(val_outputs[:, 1], wording)

        print(f"Validation Loss: {val_loss / len(val_loader)}")
    model.train()  # 다음 epoch를 위해서 다시 train 모드로 설정한다.


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Creating test loader    </div>

In [ ]:
# test dataset을 위한 test 데이터 로더를 만든다.
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Generating predictions on test set    </div>

In [ ]:
model.eval()  # 모델을 평가 모드로 전환
predictions = []  # 예측값을 저장할 빈 리스트 생성
with torch.no_grad():  # gradient 계산 x
    for input_ids, attention_mask in test_loader: # 테스트 데이터로더에서 배치 단위로 input_ids와 attention_mask를 불러온다.
        
        # GPU에 데이터 담기
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        outputs = model(input_ids, attention_mask) # 모델에 input_ids와 attention_mask 넣고 Predication 수행. outputs에는 해당 배치의 모델 prediction 값이 저장되어 있다.
        
        predictions.extend(outputs.cpu().numpy()) 
        """ 
        outputs 텐서를 cpu()로 옮기고, numpy 배열로 변환한다. 
        그 후 predictions 리스트에 이 값을 추가한다. 왜 CPU로 옮기냐? 
        .numpy()를 이용해서 Pytorch 텐서를 numpy 배열로 변환할때,
        해당 텐서는 꼭 CPU에 있어야 하기 때문이다."""



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Generating submission
    </div>

In [ ]:
submission_df = pd.DataFrame({  # 새 데이터프레임 submission_df 생성
    'student_id': test_data['student_id'],  # test_data에서 'student_id' 열 값을 가져와서, submission_df의 'student_id' 열에 그대로 할당
    'content': [pred[0] for pred in predictions],  # predictions 리스트의 각 원소에서 첫 번째 값(pred[0])을 가져와서 새로운 리스트를 만들고, 'content' 열에 할당
    'wording': [pred[1] for pred in predictions]  # predictions 리스트의 각 원소에서 두 번째 값(pred[1])을 가져와서 새로운 리스트를 만들고, 'wording' 열에 할당
})

submission_df.to_csv('submission.csv', index=False)  # dataframe 내용읠 csv파일로 저장, 행 번호는 포함하지 않도록 index=False

In [ ]:
submission_df